In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [3]:
def date_time(table_cells):
    """
    This function returns the date and time from the HTML table cell
    Input: the element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]


def booster_version(table_cells):
    """
    This function returns the booster version from the HTML table cell
    Input: the element of a table data cell extracts extra row
    """
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out


def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the element of a table data cell extracts extra row
    """
    out = [i for i in table_cells.strings][0]
    return out

In [ ]:
def get_mass(table_cells):
    """
    This function returns the payload mass from the HTML table cell
    Input: the element of a table data cell extracts extra row
    """
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg") + 2]
    else:
        new_mass = 0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the column name from the HTML table header
    Input: the element of a table header cell extracts extra row
    """
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    column_name = ' '.join(row.contents)

    # Filter out digits and empty names
    if not (column_name.strip().isdigit()):
        column_name = column_name.strip()
    return column_name

In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [ ]:
TASK 1: Request the Falcon 9 Launch Wikipedia Page
Use the provided URL to fetch the Falcon 9 launch list page using Python’s requests library.

In [ ]:
NHIỆM VỤ 1: Yêu cầu Trang Wikipedia về Phóng Falcon 9
Sử dụng URL được cung cấp để lấy trang danh sách phóng Falcon 9 bằng thư viện requests của Python.

In [5]:
# use requests.get() method with the provided static_url
# assign the response to an object

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

response = requests.get(static_url, headers=headers)
print("Status Code:", response.status_code)

Status Code: 200


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "html.parser")
print("BeautifulSoup object created successfully!")

BeautifulSoup object created successfully!


In [8]:
# Use soup.title attribute

print("Page Title:", soup.title.string)

Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [ ]:
TASK 2: Extract all column/variable names from the HTML table header
Use BeautifulSoup to parse the Falcon 9 launch table and extract all header (<th>) values as column names.

In [ ]:
NHIỆM VỤ 2: Trích xuất tất cả tên cột/biến từ tiêu đề bảng HTML
Sử dụng BeautifulSoup để phân tích bảng khởi chạy Falcon 9 và trích xuất tất cả giá trị tiêu đề (<th>) dưới dạng tên cột.

In [9]:
# Use the find_all function in the BeautifulSoup object, with element type 'table'
# Assign the result to a list called `html_tables`

html_tables = soup.find_all("table")
print(f"Number of tables found: {len(html_tables)}")

Number of tables found: 25


In [10]:
# Kiểm tra xem bảng nào chứa cột 'Flight' hoặc 'Payload'
for i, table in enumerate(html_tables):
    snippet = str(table)[:400]  # chỉ in 400 ký tự đầu
    if 'Flight' in snippet or 'Payload' in snippet:
        print(f"Table index: {i}")
        print(snippet)
        print("-" * 60)

Table index: 2
<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class=
------------------------------------------------------------
Table index: 3
<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="
------------------------------------------------------------
Table index: 4
<table class="wikitable plainro

In [12]:
# Find all tables
html_tables = soup.find_all("table")
print(f"Number of tables found: {len(html_tables)}")

# Select the confirmed Falcon 9 launch table (index 2)
first_launch_table = html_tables[2]

# Find header row (the first <tr> containing <th>)
header_row = first_launch_table.find("tr")

column_names = []
for th in header_row.find_all("th"):
    # Clean HTML tags
    if th.br:
        th.br.replace_with(" ")  # đổi <br> thành khoảng trắng
    if th.a:
        th.a.unwrap()
    if th.sup:
        th.sup.decompose()

    text = th.get_text(" ", strip=True)
    text = " ".join(text.split())  # xóa khoảng trắng thừa
    if text and not text.isdigit():
        column_names.append(text)

print("Extracted column names:")
print(column_names)

Number of tables found: 25
Extracted column names:
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [ ]:
TASK 3: Create a DataFrame by parsing the launch HTML tables
Use pandas.read_html() or BeautifulSoup to extract the Falcon 9 launch tables
from the Wikipedia page and convert them into a structured Pandas DataFrame.

In [ ]:
NHIỆM VỤ 3: Tạo DataFrame bằng cách phân tích các bảng HTML khởi chạy
Sử dụng pandas.read_html() hoặc BeautifulSoup để trích xuất các bảng khởi chạy Falcon 9
từ trang Wikipedia và chuyển đổi chúng thành một DataFrame có cấu trúc của Pandas.

In [13]:
# Create an empty list to store parsed launch records
launch_data = []

rows = first_launch_table.find_all("tr")[1:]

for row in rows:
    cells = row.find_all(["th", "td"])
    row_data = []

    for cell in cells:
        # Clean up HTML tags
        if cell.a:
            cell.a.unwrap()
        if cell.sup:
            cell.sup.decompose()
        if cell.br:
            cell.br.replace_with(" ")

        text = cell.get_text(" ", strip=True)
        text = " ".join(text.split())
        row_data.append(text)

    # Standardize number of columns to match header length
    if len(row_data) > len(column_names):
        row_data = row_data[:len(column_names)]          # cắt bớt nếu thừa
    elif len(row_data) < len(column_names):
        row_data += [""] * (len(column_names) - len(row_data))  # thêm rỗng nếu thiếu

    if len(row_data) > 0:
        launch_data.append(row_data)

print(f"Total rows extracted: {len(launch_data)}")

Total rows extracted: 15


In [14]:
# Convert the list of lists into a DataFrame
launch_df = pd.DataFrame(launch_data, columns=column_names)

print("Shape of DataFrame:", launch_df.shape)
launch_df.head()

Shape of DataFrame: (15, 8)


,Flight No.,Date and time ( ),Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome
0,1,"4 June 2010, 18:45",F9 v1.0 B0003.1 [ 8 ],"CCAFS , SLC-40",Dragon Spacecraft Qualification Unit,,LEO,SpaceX
1,First flight of Falcon 9 v1.0. Used a boilerpl...,,,,,,,
2,2,"8 December 2010, 15:43",F9 v1.0 B0004.1 [ 8 ],"CCAFS , SLC-40",Dragon demo flight C1 (Dragon C101),,LEO ( ISS ),NASA ( COTS ) NRO
3,"Maiden flight of Dragon capsule , consisting o...",,,,,,,
4,3,"22 May 2012, 07:44",F9 v1.0 B0005.1 [ 8 ],"CCAFS , SLC-40",Dragon demo flight C2+ (Dragon C102),"525 kg (1,157 lb)",LEO ( ISS ),NASA ( COTS )


In [18]:
launch_df.to_csv("falcon9_launches.csv", index=False)